<a href="https://colab.research.google.com/github/AjeetSingh02/Notebooks/blob/master/sentimentClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2019-09-23 15:50:54--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  24.3MB/s    in 3.3s    

2019-09-23 15:50:57 (24.3 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [0]:
import shutil
shutil.unpack_archive("aclImdb_v1.tar.gz", "/content/")

In [0]:
import nltk 
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import os
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [0]:
path = "/content/aclImdb/train/pos/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.readlines()[0])

first_1000 = temp[:100]
pos = pd.DataFrame({"reviews": first_1000, "labels": list(np.ones(len(first_1000), dtype=int))})

In [0]:
path = "/content/aclImdb/train/neg/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.readlines()[0])
        
first_1000 = temp[:100]
neg = pd.DataFrame({"reviews": first_1000, "labels": list(np.zeros(len(first_1000), dtype=int))})

In [0]:
train_data = pd.concat([pos, neg], ignore_index=True)

In [0]:
path = "/content/aclImdb/test/pos/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.readlines()[0])
        
first_1000 = temp[:100]
pos = pd.DataFrame({"reviews": first_1000, "labels": list(np.ones(len(first_1000), dtype=int))})

In [0]:
path = "/content/aclImdb/test/neg/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.readlines()[0])
        
first_1000 = temp[:100]
neg = pd.DataFrame({"reviews": first_1000, "labels": list(np.zeros(len(first_1000), dtype=int))})

In [0]:
test_data = pd.concat([pos, neg], ignore_index=True)

In [0]:
train_data.labels.value_counts()

1    100
0    100
Name: labels, dtype: int64

In [0]:
#This is just for clear explanation. We can do all this in very small number of steps. And surely within a single funtion with very less computational complexity

def tokenize_data(dataset):
  tokenizer = nltk.tokenize.TreebankWordTokenizer()
  
  for i in range(dataset.shape[0]): 
    dataset["reviews"][i] = tokenizer.tokenize(dataset["reviews"][i])
  return dataset

def remove_stop_words(dataset):
  stop_words = set(stopwords.words('english'))
  
  for i in range(dataset.shape[0]):
    dataset["reviews"][i] = ([token.lower() for token in dataset["reviews"][i] if token not in stop_words])
  return dataset

def normalize(dataset):
  lemmatizer = nltk.stem.WordNetLemmatizer() 

  for i in range(dataset.shape[0]):
    dataset.reviews[i] = " ".join([lemmatizer.lemmatize(token) for token in dataset.reviews[i]]).strip()
  return dataset

def remove_garbage(dataset):
    garbage = "~`!@#$%^&*()_-+={[}]|\:;'<,>.?/"
    for i in range(dataset.shape[0]):
      dataset.reviews[i] = "".join([char for char in dataset.reviews[i] if char not in garbage])
    return dataset

In [0]:
train_data = tokenize_data(train_data)
train_data = remove_stop_words(train_data)
train_data = normalize(train_data)
train_data = remove_garbage(train_data)

test_data = tokenize_data(test_data)
test_data = remove_stop_words(test_data)
test_data = normalize(test_data)
test_data = remove_garbage(test_data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [0]:
train_data.head()

,reviews,labels
0,gb wrote screen adaptation nobel prize winning...,1
1,i stopped short giving mr blandings build his...,1
2,an excellent film great performance zack loch...,1
3,gorgeous techicolor production telling unusual...,1
4,in don siegel s 1971 masterpiece dirty harry ...,1


In [0]:
test_data.head()

,reviews,labels
0,yes maybe part film require suspending belief...,1
1,let s get flaw way right top movie much longe...,1
2,i first viewed movie first came also bought lp...,1
3,i finished double feature night an american we...,1
4,this funny even fell apart little end doe go ...,1


In [0]:
train_data.reviews[5]

'while soundtrack bit dated  story relevant us ever with blue collar job everyone s job outsourced us corporation government profit american suffer  br    br   peter strauss emory  steel worker work job father 35 years his wife wellportrayed pamela reed  realistic  trying support family two child emory loses job the mill closed pretext mismanagement  also embezzlement cheaper wage pay one steelworker one month  outsourcing  would pay straussemory day never mind men good loyal worker value try best family  br    br   john goodman  gary cole  strauss  brother  coworker also affected it disturbing realistic some scene emory father moving emory hope local union able reopen mill  promise so  br    br   emory s brother  lee already see writing wall there job left rustbelt  ohio  must move on however us move   where better bluecollar steel worker    br    br   there triumphant scene end emory crew fill loading dock steel products the guard allows final gesture  one men committed suicide empat

In [0]:
def fit_corpus(train_data, test_data):
  corpus = pd.DataFrame({"reviews": train_data["reviews"]})
  corpus.reviews.append(test_data["reviews"], ignore_index=True)
  tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,2))
  tfidf.fit(corpus["reviews"])
  return tfidf
  
def transform_data(tfidf, dataset):
  features = tfidf.transform(dataset["reviews"])
  return pd.DataFrame(features.todense(), columns = tfidf.get_feature_names())

In [0]:
tfidf = fit_corpus(train_data, test_data)
train_features = transform_data(tfidf, train_data)
test_features = transform_data(tfidf, test_data)

train_labels = train_data["labels"]
test_labels = test_data["labels"]

In [0]:
train_features.head()

,00,10,10 little,1010,12,13,15,19,1930s,1978,1980s,1986,1987,1989,20,2003,2005,21,24,30,30 minute,35,45,65,710,80,810,90,911,abandoned,ability,able,about,about so,absolute,absolutely,absurd,absurdity,abuse,abusive,...,wrist,write,writer,writes,writing,written,wrong,wrote,yeah,year,year ago,year late,year later,year old,years,years the,yell,yes,yesterday,yet,yet another,york,you,you find,you know,you ll,you ve,young,young lady,young man,young people,young woman,younger,your,your heart,yourself,youth,zack,zero,zombie
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094648,0.0,0.048933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1,0.0,0.0,0.0,0.125623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108893,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
2,0.0,0.0,0.0,0.151378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.60551,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.06145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.051872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043427,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.025335,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


In [0]:
clf = LogisticRegression(random_state=0, solver='lbfgs')
clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
clf.predict(test_features.iloc[[0]])

array([1])

In [0]:
test_features.iloc[[5]]

,00,10,10 little,1010,12,13,15,19,1930s,1978,1980s,1986,1987,1989,20,2003,2005,21,24,30,30 minute,35,45,65,710,80,810,90,911,abandoned,ability,able,about,about so,absolute,absolutely,absurd,absurdity,abuse,abusive,...,wrist,write,writer,writes,writing,written,wrong,wrote,yeah,year,year ago,year late,year later,year old,years,years the,yell,yes,yesterday,yet,yet another,york,you,you find,you know,you ll,you ve,young,young lady,young man,young people,young woman,younger,your,your heart,yourself,youth,zack,zero,zombie
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
train_data.reviews[5]

'while soundtrack bit dated  story relevant us ever with blue collar job everyone s job outsourced us corporation government profit american suffer  br    br   peter strauss emory  steel worker work job father 35 years his wife wellportrayed pamela reed  realistic  trying support family two child emory loses job the mill closed pretext mismanagement  also embezzlement cheaper wage pay one steelworker one month  outsourcing  would pay straussemory day never mind men good loyal worker value try best family  br    br   john goodman  gary cole  strauss  brother  coworker also affected it disturbing realistic some scene emory father moving emory hope local union able reopen mill  promise so  br    br   emory s brother  lee already see writing wall there job left rustbelt  ohio  must move on however us move   where better bluecollar steel worker    br    br   there triumphant scene end emory crew fill loading dock steel products the guard allows final gesture  one men committed suicide empat

In [0]:
print(f"The score for complete test reviews is: {clf.score(test_features, test_labels) * 100 } %")

The score for complete test reviews is: 81.5 %
